# Pre-processing techniques

In [1]:
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('../../data/final_features_df.csv')
df.head()

,Unnamed: 0,Age,Income,faves_pca0,faves_pca1,unfaves_pca0,unfaves_pca1,accessories,alcohol,animamted,...,Drama.2,Entertainment (Variety Shows),Factual,Learning,Music,News,Religion &amp; Ethics,Sport.1,Weather,Rating_bin
0,0,62,1,-0.321485,0.0786,-0.19967,-0.200645,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
1,1,62,1,-0.321485,0.0786,-0.19967,-0.200645,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
2,2,62,1,-0.321485,0.0786,-0.19967,-0.200645,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
3,3,62,1,-0.321485,0.0786,-0.19967,-0.200645,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
4,4,62,1,-0.321485,0.0786,-0.19967,-0.200645,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0


In [3]:
df_0 = df.fillna(0)

In [4]:
# df = df_0.sample(frac = 1)
# train_frac = 0.8
# test_frac = 0.1

# X_train = df[[c for c in df.columns if c != 'Rating_bin']].iloc[:int(len(df) * train_frac)].values
# y_train = df.Rating_bin.iloc[:int(len(df) * train_frac)].values

# X_test = df[[c for c in df.columns if c != 'Rating_bin']].iloc[int(len(df) * train_frac):int(len(df) * (train_frac+test_frac))].values
# y_test = df.Rating_bin.iloc[int(len(df) * train_frac):int(len(df) * (train_frac+test_frac))].values

# X_valid = df[[c for c in df.columns if c != 'Rating_bin']].iloc[int(len(df) * (train_frac+test_frac)):].values
# y_valid = df.Rating_bin.iloc[int(len(df) * (train_frac+test_frac)):].values

In [5]:
# pd.DataFrame(X_train).to_csv('../../data/X_train.csv')
# pd.DataFrame(y_train).to_csv('../../data/y_train.csv')

# pd.DataFrame(X_test).to_csv('../../data/X_test.csv')
# pd.DataFrame(y_test).to_csv('../../data/y_test.csv')

# pd.DataFrame(X_valid).to_csv('../../data/X_valid.csv')
# pd.DataFrame(y_valid).to_csv('../../data/y_valid.csv')

In [6]:
X = df_0.drop(columns='Rating_bin')
y = df_0['Rating_bin']
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=42)

In [7]:
actual_test = X_val.copy()
actual_test['true_Rating_bin'] = y_val
actual_test.shape

(9030, 515)

inspired by: https://github.com/bryantruong/examingBiasInAI/blob/main/biasInMachineLearning.ipynb

In [8]:
male_df = actual_test[actual_test['Gender_M'] == 1]
num_of_priviliged = male_df.shape[0]
female_df = actual_test[actual_test['Gender_M'] == 0]
num_of_unpriviliged = female_df.shape[0]

In [9]:
unpriviliged_outcomes = female_df[female_df['true_Rating_bin'] == 1].shape[0]
unpriviliged_ratio = unpriviliged_outcomes/num_of_unpriviliged
unpriviliged_ratio

0.13474081702711982

In [10]:
priviliged_outcomes = male_df[male_df['true_Rating_bin'] == 1].shape[0]
priviliged_ratio = priviliged_outcomes/num_of_priviliged
priviliged_ratio

0.1466916354556804

In [11]:
# Calculating disparate impact
disparate_impact = unpriviliged_ratio / priviliged_ratio
print("Disparate Impact, Sex vs. Rating: " + str(disparate_impact))

Disparate Impact, Sex vs. Rating: 0.91853101649977


## Baseline model: Decision Tree

In [12]:
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_val)
print(classification_report(y_val, y_pred))
confusion_matrix(y_val, y_pred)

              precision    recall  f1-score   support

           0       0.90      0.91      0.90      7775
           1       0.40      0.39      0.39      1255

    accuracy                           0.83      9030
   macro avg       0.65      0.65      0.65      9030
weighted avg       0.83      0.83      0.83      9030



array([[7047,  728],
       [ 769,  486]])

In [13]:
y_pred = clf.predict(X_val)
x_test_orig = X_val.copy()
x_test_orig['pred_baseline'] = y_pred
baseline_output = x_test_orig

In [14]:
male_df = baseline_output[baseline_output['Gender_M'] == 1]
num_of_priviliged = male_df.shape[0]
female_df = baseline_output[baseline_output['Gender_M'] == 0]
num_of_unpriviliged = female_df.shape[0]

In [15]:
unpriviliged_outcomes = female_df[female_df['pred_baseline'] == 1].shape[0]
unpriviliged_ratio = unpriviliged_outcomes/num_of_unpriviliged
unpriviliged_ratio

0.12615859938208032

In [16]:
priviliged_outcomes = male_df[male_df['pred_baseline'] == 1].shape[0]
priviliged_ratio = priviliged_outcomes/num_of_priviliged
priviliged_ratio

0.14950062421972535

In [17]:
# Calculating disparate impact
disparate_impact = unpriviliged_ratio / priviliged_ratio
print("Disparate Impact, Sex vs. Predicted Rating: " + str(disparate_impact))

Disparate Impact, Sex vs. Predicted Rating: 0.8438667065139569


## Reweight

In [18]:
from aif360.datasets import StandardDataset
from aif360.algorithms.preprocessing import Reweighing, DisparateImpactRemover
from aif360.metrics import BinaryLabelDatasetMetric

pip install 'aif360[LawSchoolGPA]'
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'


In [19]:
privileged_groups = [{'Gender_M': 1}]
unprivileged_groups = [{'Gender_M': 0}]

In [20]:
df_0['Rating_bin'].value_counts()

0    31279
1     4841
Name: Rating_bin, dtype: int64

In [21]:
aif360_df = StandardDataset(
    df = df_0.drop('Gender_F', axis = 1),
    label_name = 'Rating_bin',
    protected_attribute_names = ['Gender_M'],
    favorable_classes = [0],
    privileged_classes = [df_0['Gender_M'], lambda x: x == 1]
)

In [22]:
df_train, df_test = aif360_df.split([0.7], shuffle=True)

In [23]:
RW = Reweighing(unprivileged_groups=unprivileged_groups,
               privileged_groups=privileged_groups)

In [24]:
RW.fit(df_train)
dataset_transf = RW.transform(df_train)

In [25]:
X = dataset_transf.features
y = dataset_transf.labels
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=42)

In [26]:
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_val)
print(classification_report(y_val, y_pred))
confusion_matrix(y_val, y_pred)

              precision    recall  f1-score   support

         0.0       0.91      0.91      0.91      5499
         1.0       0.39      0.39      0.39       822

    accuracy                           0.84      6321
   macro avg       0.65      0.65      0.65      6321
weighted avg       0.84      0.84      0.84      6321



array([[5004,  495],
       [ 505,  317]])

In [27]:
metric_orig = BinaryLabelDatasetMetric(df_train, 
                                       unprivileged_groups=unprivileged_groups,
                                       privileged_groups=privileged_groups)

print("Difference in mean outcomes between unprivileged and privileged groups in original dataset = %f" % metric_orig.mean_difference())

Difference in mean outcomes between unprivileged and privileged groups in original dataset = 0.012661


In [28]:
metric_orig = BinaryLabelDatasetMetric(dataset_transf, 
                                       unprivileged_groups=unprivileged_groups,
                                       privileged_groups=privileged_groups)

print("Difference in mean outcomes between unprivileged and privileged groups in transformed dataset = %f" % metric_orig.mean_difference())

Difference in mean outcomes between unprivileged and privileged groups in transformed dataset = 0.000000


## Disparate Impact Remover

In [35]:
di = DisparateImpactRemover(repair_level = 1.0)
dataset_transf_train = di.fit_transform(df_train)
transformed = dataset_transf_train.convert_to_dataframe()[0]

In [36]:
X = dataset_transf_train.features
y = dataset_transf_train.labels
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=42)

In [37]:
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_val)
print(classification_report(y_val, y_pred))
confusion_matrix(y_val, y_pred)

              precision    recall  f1-score   support

         0.0       0.91      0.91      0.91      5499
         1.0       0.40      0.38      0.39       822

    accuracy                           0.85      6321
   macro avg       0.65      0.65      0.65      6321
weighted avg       0.84      0.85      0.84      6321



array([[5031,  468],
       [ 509,  313]])

In [38]:
X = transformed.drop('Rating_bin', axis = 1)
y = transformed['Rating_bin']
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=42)

In [39]:
x_test_transf = X_val.copy()
y_pred = clf.predict(X_val)
x_test_transf['pred_model'] = y_pred
model_output = x_test_transf

/Users/juliatessler/Unicamp/mo810/.venv/lib/python3.10/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(


In [42]:
male_df = model_output[model_output['Gender_M'] == 1]
n_priviledged = male_df.shape[0]

female_df = model_output[model_output['Gender_M'] == 0]
n_unpriviledged = female_df.shape[0]

In [43]:
unpriviliged_outcomes = female_df[female_df['pred_model'] == 1].shape[0]
unpriviliged_ratio = unpriviliged_outcomes/num_of_unpriviliged
unpriviliged_ratio

0.08067284586337109

In [44]:
disparate_impact = unpriviliged_ratio / priviliged_ratio
print("Disparate Impact, Sex vs. model output: " + str(disparate_impact))

Disparate Impact, Sex vs. model output: 0.5396154449817139
